In [ ]:
from openai import OpenAI
from math import exp
import numpy as np
from IPython.display import display, HTML
import os

# load dotenv
from dotenv import load_dotenv
load_dotenv()

client = OpenAI(
    api_key=os.environ.get(
        "OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"
    )
)

In [ ]:
def get_completion(
    messages: list[dict[str, str]],
    model: str = "gpt-4o",
    temperature=0,
    stop=None,
    seed=49,
    tools=None,
    logprobs=None,  # whether to return log probabilities of the output tokens or not. If true, returns the log probabilities of each output token returned in the content of message..
    top_logprobs=None,
) -> str:
    params = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
        "stop": stop,
        "seed": seed,
        "logprobs": logprobs,
        "top_logprobs": top_logprobs,
    }
    if tools:
        params["tools"] = tools

    completion = client.chat.completions.create(**params)
    return completion

In [ ]:
high_prob_completions = {}
low_prob_completions = {}
html_output = ""

sentence_list = [
    """
Aircraft idx: ['FLIGHT1', 'FLIGHT2']
Info on FLIGHT1 B777 index = 0
Pos: N84'48'8.0"  E129'53'19.0"
Hdg: 005   Trk: 005
Alt: 27400 ft  V/S: 0 fpm
CAS/TAS/GS: 179/274/274 kts   M: 0.460

Info on FLIGHT2 A320 index = 1
Pos: N84'37'33.0"  E129'43'5.0"
Hdg: 005   Trk: 005
Alt: 27400 ft  V/S: 0 fpm
CAS/TAS/GS: 237/359/359 kts   M: 0.602


### Actionable Plan to Resolve Conflicts:
1. FLIGHT2: descend to 
"""
]

for sentence in sentence_list:
    html_output += f"<p>Sentence: {sentence}</p>"

    for temperature in [0.0, 0.6, 1.2, 1.8]:
        PROMPT = """Complete this sentence. You are acting as auto-complete. Simply complete the sentence to the best of your ability  {sentence}"""
        API_RESPONSE = get_completion(
            [{"role": "user", "content": PROMPT.format(sentence=sentence)}],
            model="gpt-4o",
            logprobs=True,
            top_logprobs=6,
            temperature=temperature,
        )

        html_output += f"<p><strong>Temperature: {temperature}</strong></p>"
        first_token = True
        for token in API_RESPONSE.choices[0].logprobs.content[0].top_logprobs:
            html_output += f'<p style="color:cyan">Predicted next token: {token.token}, <span style="color:darkorange">logprobs: {token.logprob}, <span style="color:magenta">linear probability: {np.round(np.exp(token.logprob)*100,2)}%</span></p>'
            if first_token:
                if np.exp(token.logprob) > 0.95:
                    high_prob_completions[sentence] = token.token
                if np.exp(token.logprob) < 0.60:
                    low_prob_completions[sentence] = token.token
            first_token = False
        html_output += "<br>"

display(HTML(html_output))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
matplotlib.rc("font", size=12, family="Ubuntu")
matplotlib.rc("lines", linewidth=2, markersize=8)
matplotlib.rc("grid", color="darkgray", linestyle=":")


# Data
temp_0_tokens = ["260", "250", "1", "270", "26", "25"]
temp_0_probs = [53.88, 9.36, 7.29, 6.44, 5.01, 4.42]

temp_06_tokens = ["260", "270", "26", "250", "25", "262"]
temp_06_probs = [48.71, 15.81, 5.82, 4.53, 4.0, 2.75]

# Create the plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 10))
fig.suptitle("FLIGHT2: descend to ____", fontsize=16)

# Temperature 0.0
ax1.bar(temp_0_tokens, temp_0_probs, color="tab:blue")
ax1.set_title("Temperature: 0.0")
ax1.set_xlabel("Tokens")
ax1.set_ylabel("Probability (%)")
ax1.set_ylim(0, 60)

# Temperature 0.6
ax2.bar(temp_06_tokens, temp_06_probs, color="tab:red")
ax2.set_title("Temperature: 0.6")
ax2.set_xlabel("Tokens")
ax2.set_ylabel("Probability (%)")
ax2.set_ylim(0, 60)

# Remove grid lines
ax1.grid(False)
ax2.grid(False)

plt.tight_layout()
plt.savefig(
    "../results/figures2/token_vs_temperature.pdf", format="pdf", bbox_inches="tight"
)

plt.show()